## Agent-Based Code Execution using Amazon AgentCore Bedrock Code Interpreter- Tutorial(Strands)
This tutorial demonstrates how to create an AI agent that validates answers through code execution using Python. We use Amazon Bedrock AgentCore Code Interpreter to run code that is generated by the LLM

This tutorial demonstrates how to use AgentCore Bedrock Code Interpreter to:
1. Set up a sandbox environment
2. Configure a strands based agent that generated code based on the user query
3. Execute code in a sandbox environment using Code Interpreter
4. Display the results back to the user

## Prerequisites
- AWS account with Bedrock AgentCore Code Interpreter access
- You have the necessary IAM permissions to create and manage code interpreter resources
- Required Python packages installed(including boto3, bedrock-agentcore & strands)
- IAM role should have permissions to invoke models on Amazon Bedrock
 - Access to Claude 3.7 Sonnet model in the US Oregon (us-west-2) region (default model for Strands SDK)

## Your IAM execution role should have the following IAM policy attached



~~~ {
"Version": "2012-10-17",
"Statement": [
    {
        "Effect": "Allow",
        "Action": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "Resource": "*"
    },
    {
        "Effect": "Allow",
        "Action": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
}

## How it works

The code execution sandbox enables agents to safely process user queries by creating an isolated environment with a code interpreter, shell, and file system. After a Large Language Model helps with tool selection, code is executed within this session, before being returned to the user or Agent for synthesis.

![architecture local](code-interpreter.png)

## 1. Setting Up the Environment

First, let's import the necessary libraries and initialize our Code Interpreter session.

In [ ]:
!pip install --upgrade -r requirements.txt

In [2]:
from bedrock_agentcore.tools.code_interpreter_client import code_session
from strands import Agent, tool
import json
from typing import Dict, Any, List


## 2.System Prompt Definition
Define the behavior and capabilities of the AI assistant. We instruct our assistant to always validate answers through code execution and data based reasoning.

In [3]:
SYSTEM_PROMPT = """You are a helpful AI assistant that validates all answers through code execution.

VALIDATION PRINCIPLES:
1. When making claims about code, algorithms, or calculations - write code to verify them
2. Use execute_python to test mathematical calculations, algorithms, and logic
3. Create test scripts to validate your understanding before giving answers
4. Always show your work with actual code execution
5. If uncertain, explicitly state limitations and validate what you can

APPROACH:
- If asked about a programming concept, implement it in code to demonstrate
- If asked for calculations, compute them programmatically AND show the code
- If implementing algorithms, include test cases to prove correctness
- Document your validation process for transparency
- The sandbox maintains state between executions, so you can refer to previous results

TOOL AVAILABLE:
- execute_python: Run Python code and see output

RESPONSE FORMAT: The execute_python tool returns a JSON response with:
- sessionId: The sandbox session ID
- id: Request ID
- isError: Boolean indicating if there was an error
- content: Array of content objects with type and text/data
- structuredContent: For code execution, includes stdout, stderr, exitCode, executionTime"""

## 3.Code Execution Tool Definition
Next we define the function as tool that will be used by the Agent as tool, to run code in the code sandbox. We use the @tool decorator to annotate the function as a custom tool for the Agent.

Within an active code interpreter session, you can execute code in supported languages (Python, JavaScript), access libraries based on your dependencies configuration, generate visualizations, and maintain state between executions.

In [4]:
@tool
def execute_python(code: str, description: str = "") -> str:
    """Execute Python code in the sandbox."""
    
    if description:
        code = f"# {description}\n{code}"
    
    print(f"\n Generated Code: {code}")
    
    with code_session("us-west-2") as code_client:
        response = code_client.invoke("executeCode", {
            "code": code,
            "language": "python",
            "clearContext": False
        })
    
    for event in response["stream"]:
        return json.dumps(event["result"])

## 4. Agent Configuration
We create and configure an agent using the Strands SDK.We provide it the system prompt and the tool we defined above to execute generate code

In [5]:
agent = Agent(
    tools=[execute_python],
    system_prompt=SYSTEM_PROMPT,
    callback_handler=None
)

## 5. Query Definition
Define a sample query to test the agent with code execution capabilities

In [6]:
query = "Tell me the largest random prime number between 1 and 100, which is less than 84 and more that 9"

## 6. Agent Invocation and Response Processing
We invoke the agent with our query and process the agent's response


Note: Async execution requires running in an async environment

In [7]:
try:
    response_text = ""
    async for event in agent.stream_async(query):
        if "data" in event:
            chunk = event["data"]
            response_text += chunk
            print(chunk, end="")
except Exception as e:
    print(f"Error occurred: {str(e)}")

I'll help you find the largest prime number between 1 and 100 that is less than 84 and greater than 9. Let me write code to identify all prime numbers in that range and find the largest one.
 Generated Code: def is_prime(n):
    """Check if a number is prime"""
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    
    # Check odd divisors up to sqrt(n)
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True

# Find all prime numbers between 1 and 100 that are > 9 and < 84
primes_in_range = []
for num in range(10, 84):  # 10 to 83 (inclusive)
    if is_prime(num):
        primes_in_range.append(num)

print("Prime numbers between 9 and 84:")
print(primes_in_range)
print(f"\nLargest prime in this range: {max(primes_in_range)}")
Let me verify that 83 is indeed prime by double-checking our calculation:
 Generated Code: # Verify that 83 is prime
def verify_prime(n):
    """Detailed